In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
#import cudf
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

sns.set_theme()
sns.set_style('dark')
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/ventilator-pressure-prediction/train.csv',index_col='id')
test = pd.read_csv('/kaggle/input/ventilator-pressure-prediction/test.csv')
print(f' Shape of the train data : {train.shape}')
print(f' Shape of the test data : {test.shape}')


In [ ]:
train.info();

In [ ]:
train.head()

In [ ]:
train['breath_id'].value_counts()

In [ ]:
print(f'Each breath_id has 80 entries ?{80*train["breath_id"].nunique() == train.shape[0]}')

In [ ]:
train.isna().sum()

### Let us look at any 1 breath_id in more detail

In [ ]:
breath_id1=train[train.breath_id ==1]
breath_id2 = train[train['breath_id'] == 2]
breath_id3 = train[train['breath_id'] == 3]

In [ ]:
plt=sns.relplot(kind='scatter',data=breath_id2,x='time_step',y='pressure')

In [ ]:
plt=sns.relplot(kind='line',data=breath_id3,x='time_step',y='pressure')

In [ ]:
sns.set_palette("Set2")

In [ ]:
plt = sns.catplot(data=train,kind='box',x='pressure',row='C',col='R')

In [ ]:
g= sns.FacetGrid(train,col='R',row='C',sharex=True)
g.map_dataframe(sns.scatterplot,'time_step','pressure')
g.refline(y=train["pressure"].median(),color='black')
g.refline(y=train["pressure"].mean(),color='red')

In [ ]:
g= sns.FacetGrid(train,col='R',row='C',sharex=True)
g.map_dataframe(sns.scatterplot,'u_in','pressure')
g.refline(y=train["pressure"].median(),color='black')
g.refline(y=train["pressure"].mean(),color='red')

#### The columns C, R and u_out can be treated as categorical columns

In [ ]:
cateorical_columns = ['breath_id','C','R','u_out']
numerical_columns=['time_step','u_in']
train['breath_id'].nunique(),train['u_in'].nunique()

In [ ]:
plt=sns.kdeplot(data=train[train['breath_id'].isin(list(range(1,500)))],y='pressure',x='time_step',alpha=0.7)

In [ ]:
sns.set()
ecdf_plot=sns.ecdfplot(train.sort_values('pressure'), 
                               x="pressure",marker='.',linestyle='none')
percentiles = np.array([2.5,25,50,75,97.5])
pressure_ptiles= np.percentile(train['pressure'],percentiles)
ecdf_plot.plot(pressure_ptiles, percentiles/100, marker='D', color='red',linestyle='none',)


In [ ]:
sns.set_style('whitegrid')
timestep_ecdf_plot=sns.displot(train,x='time_step',y='pressure',col='C',row='R')
timestep_ecdf_plot.refline(y=train["pressure"].median(),color='black')
timestep_ecdf_plot.refline(y=train["pressure"].mean(),color='red')
timestep_ecdf_plot.refline(y=train["pressure"].std(),color='orange')
timestep_ecdf_plot.refline(y=train["pressure"].min(),color='blue')

In [ ]:
print(f'The number of records where pressure recorded < 0 : {train.query("(pressure <0)").shape[0]}')

In [ ]:
train.query("(pressure <0)").groupby('breath_id')['breath_id'].agg('count')

In [ ]:
train.groupby('breath_id').agg({'time_step':['mean','min','max']})

In [ ]:
series = train.groupby('breath_id').collect().reset_index()
exhale = 80 - train.groupby('breath_id')[['u_out']].agg('sum')

In [ ]:
train[train['breath_id'] == 70898]['u_out'].value_counts()

In [ ]:
pd_train = pd.read_csv('/kaggle/input/ventilator-pressure-prediction/train.csv')


In [ ]:
np.arange(80)

In [ ]:
np.linspace(0,80,1)

In [ ]:
#x = np.linspace(0,3,80)
x = train[train['breath_id'] == 58]['time_step']
pressure= train[train['breath_id'] == 58]['pressure']
u_in = train[train['breath_id'] == 58]['u_in']
u_out = train[train['breath_id'] == 58]['u_out']
plt.figure(figsize=(15,5))
ax = plt.plot(x,pressure,marker='.',label='pressure')
plt.plot(x,u_in,marker='.',label='u_in')
plt.plot(x,u_out,marker='.',label='u_out')
plt.legend(loc='upper right')
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
x = train[train['breath_id'] == 87]['time_step']
for i in range(1,5000):
    pressure= train[train['breath_id'] == i]['pressure']
    if not pressure.empty:
        ax = plt.plot(x,pressure,color='blue')
        u_in = train[train['breath_id'] == i]['u_in']
        plt.plot(x,u_in,color='red')
        u_out = train[train['breath_id'] == i]['u_out']
        plt.plot(x,u_out,color='orange')

plt.legend(['pressure','u_in','u_out'],loc='upper right',)
plt.show()

In [ ]:
train['u_in'].min(),train['u_in'].max()

In [ ]:
breath_id_byCandR = train.groupby(['C','R'])['breath_id'].unique()

In [ ]:
breath_id_byCandR.keys()

In [ ]:
plt.figure(figsize=(15,5))
x = train[train['breath_id'] == 87]['time_step']
breathids = breath_id_byCandR[(10,  5)]
for breath_id in breathids[:300]:
    pressure= train[train['breath_id'] == breath_id]['pressure']
    ax = plt.plot(x,pressure,color='blue')
    u_in = train[train['breath_id'] == breath_id]['u_in']
    plt.plot(x,u_in,color='red')
    u_out = train[train['breath_id'] == breath_id]['u_out']
    plt.plot(x,u_out,color='orange')

plt.legend(['pressure','u_in','u_out'],loc='upper right',)
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
x = train[train['breath_id'] == 87]['time_step']
breathids = breath_id_byCandR[(10, 20)]
for breath_id in breathids[:300]:
    pressure= train[train['breath_id'] == breath_id]['pressure']
    ax = plt.plot(x,pressure,color='blue')
    u_in = train[train['breath_id'] == breath_id]['u_in']
    plt.plot(x,u_in,color='red')
    u_out = train[train['breath_id'] == breath_id]['u_out']
    plt.plot(x,u_out,color='orange')

plt.legend(['pressure','u_in','u_out'],loc='upper right',)
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
x = train[train['breath_id'] == 87]['time_step']
breathids = breath_id_byCandR[(20, 20)]
for breath_id in breathids[:300]:
    pressure= train[train['breath_id'] == breath_id]['pressure']
    ax = plt.plot(x,pressure,color='blue')
    u_in = train[train['breath_id'] == breath_id]['u_in']
    plt.plot(x,u_in,color='red')
    u_out = train[train['breath_id'] == breath_id]['u_out']
    plt.plot(x,u_out,color='orange')

plt.legend(['pressure','u_in','u_out'],loc='upper right',)
plt.show()

In [ ]:
#plt.figure(figsize=(15,45))
fig,axs = plt.subplots(9,1,figsize=(15,45))
x = train[train['breath_id'] == 87]['time_step']
for i,key in enumerate(breath_id_byCandR.keys()):
    
    breathids = breath_id_byCandR[key]
    for breath_id in breathids[:200]:
        pressure= train[train['breath_id'] == breath_id]['pressure']
        axs[i].plot(x,pressure,color='blue')
        u_in = train[train['breath_id'] == breath_id]['u_in']
        axs[i].plot(x,u_in,color='red')
        u_out = train[train['breath_id'] == breath_id]['u_out']
        axs[i].plot(x,u_out,color='orange')
        title = f'C = {key[0]}, R = {key[1]}'
        axs[i].set_title(title)
        axs[i].legend(['pressure','u_in','u_out'],loc='upper right',)
plt.show()